# Temporal Features - RNN Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import sqlalchemy
from sqlalchemy import create_engine, inspect

import math
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import sys
import time
from datetime import datetime
from contextlib import redirect_stdout

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

## Import datasets

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'delta',
 'deltachroma1',
 'deltachroma10',
 'deltachroma11',
 'deltachroma12',
 'deltachroma2',
 'deltachroma3',
 'deltachroma4',
 'deltachroma5',
 'deltachroma6',
 'deltachroma7',
 'deltachroma8',
 'deltachroma9',
 'deltachromastd',
 'deltaenergy',
 'deltaenergyentropy',
 'deltamfcc1',
 'deltamfcc10',
 'deltamfcc11',
 'deltamfcc12',
 'deltamfcc13',
 'deltamfcc2',
 'deltamfcc3',
 'deltamfcc4',
 'deltamfcc5',
 'deltamfcc6',
 'deltamfcc7',
 'deltamfcc8',
 'deltamfcc9',
 'deltaspectralcentroid',
 'deltaspectralentropy',
 'deltaspectralflux',
 'deltaspectralrolloff',
 'deltaspectralspread',
 'deltazcr',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',

In [4]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

## Preprocessing

### Define the target variable

In [5]:
# Isolate the diagnosis column
y = dataframes['diagnosis_df']['diagnosis'].copy()

# Encode the target variable, ignore subtype
y = y.apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Recombine the feature variables

In [6]:
# Define non-temporal features
non_temporal = [
    'rval_df', 'gval_df', 'bval_df', 'aval_df',
    'demographic_df', 'diagnosis_df', 'habits_df'
]

# Define subset of only temporal features
temporal_tables = list(set(dataframes.keys()).difference(set(non_temporal)))

# Initialise a dictionary to hold all features
all_feats = dict()

# Loop through each temporal feature
for table in sorted(temporal_tables):
    
    # Get the dataframe
    df = dataframes[table]
    
    # Get the feature array and voice IDs
    feat_array = df.values[:, 1:]
    voice_list = df.values[:, 0]
    
    # Initialise a feature dictionary
    feat_dict = dict()
    
    # Use a for-loop to populate the dictionary
    for idx, feat in enumerate(feat_array):
        feat_dict[voice_list[idx]] = feat
    
    # Append to the all feature dictionary
    feat_name = table.split("_")[0]
    all_feats[feat_name] = feat_dict

# Convert to a dataframe
feat_df = pd.DataFrame(all_feats)
feat_df.head()

,chroma10,chroma11,chroma12,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,mfcc6,mfcc7,mfcc8,mfcc9,spectralcentroid,spectralentropy,spectralflux,spectralrolloff,spectralspread,zcr
voice094,"[8.812940625412099e-36, 8.812940625412099e-36,...","[0.0, 0.0, 0.0, 0.0147606720333849, 0.01584454...","[0.0, 0.0, 0.0, 0.0053144218490188, 0.00087902...","[0.0, 0.0, 0.0, 0.0187836972035133, 0.00990425...","[9.309714538072168e-37, 9.309714538072168e-37,...","[0.1999999999999999, 0.1999999999999999, 0.199...","[4.375660276485931e-36, 4.375660276485931e-36,...","[4.1989525084407034e-36, 4.1989525084407034e-3...","[0.0, 0.0, 0.0, 0.0039407474502982, 0.00375010...","[1.0843174854821264e-34, 1.0843174854821264e-3...",...,"[2.2282290724039535e-08, 2.2282290724039535e-0...","[9.550392832100816e-09, 9.550392832100816e-09,...","[1.0997317272915503e-09, 1.0997317272915503e-0...","[-6.646838331822851e-09, -6.646838331822851e-0...","[0.005, 0.005, 0.005, 0.4418387270131182, 0.43...","[1.3265869665663234e-10, 1.3265869665663234e-1...","[0.0, 0.0, 0.0, 0.982396648248422, 0.002750641...","[0.0, 0.0, 0.0, 0.83, 0.82, 0.8, 0.8, 0.795, 0...","[3.2557795197439587e-09, 3.2557795197439587e-0...","[0.0, 0.0, 0.0, 0.1102756892230576, 0.29323308..."
voice080,"[0.0, 0.0, 0.0, 0.0217640921596195, 0.02174452...","[0.0, 0.0, 0.0, 0.0156245128625292, 0.01738656...","[0.0, 0.0, 0.0, 0.0043743300147374, 0.00075576...","[0.0, 0.0, 0.0, 0.0107102441077124, 0.00191337...","[0.0, 0.0, 0.0, 0.0025800560593749, 0.00096684...","[0.2, 0.2, 0.2, 0.0126711230005733, 0.00269492...","[0.0, 0.0, 0.0, 0.0097001241421853, 0.00739546...","[0.0, 0.0, 0.0, 0.0053029470956855, 0.00195036...","[0.0, 0.0, 0.0, 0.0041761330807683, 0.00268102...","[0.0, 0.0, 0.0, 0.0164927773432678, 0.00271966...",...,"[0.0, 0.0, 0.0, 0.3993320411338776, 0.05136695...","[0.0, 0.0, 0.0, 0.0632751830849659, 0.48181304...","[0.0, 0.0, 0.0, -0.3106478561469513, -0.523495...","[0.0, 0.0, 0.0, -0.0575526595349513, -0.019965...","[0.0049999999999999, 0.0049999999999999, 0.004...","[1.1908465736187648e-10, 1.1908465736187648e-1...","[0.0, 0.0, 0.0, 0.9994814676223645, 0.00566532...","[0.0, 0.0, 0.0, 0.355, 0.34, 0.345, 0.345, 0.3...","[8.88178419700125e-19, 8.88178419700125e-19, 8...","[0.0, 0.0, 0.0, 0.0601503759398496, 0.16040100..."
voice057,"[3.325996765507969e-36, 3.325996765507969e-36,...","[0.0, 0.0, 0.0, 0.01171374859289, 0.0169411916...","[0.0, 0.0, 0.0, 0.0075471453691141, 0.02342467...","[0.0, 0.0, 0.0, 0.0129135358652838, 0.00069877...","[3.513478844070012e-37, 3.513478844070012e-37,...","[0.2, 0.2, 0.2, 0.0166678406357805, 0.00341123...","[1.651370699649231e-36, 1.651370699649231e-36,...","[1.584681328877364e-36, 1.584681328877364e-36,...","[0.0, 0.0, 0.0, 0.0191718039746937, 0.01075560...","[4.0922055449893015e-35, 4.0922055449893015e-3...",...,"[1.1141149167730065e-08, 1.1141149167730065e-0...","[4.775199217216706e-09, 4.775199217216706e-09,...","[5.498671368793281e-10, 5.498671368793281e-10,...","[-3.323422009673822e-09, -3.323422009673822e-0...","[0.005, 0.005, 0.005, 0.2377797344583633, 0.22...","[2.0026124684413918e-10, 2.0026124684413918e-1...","[0.0, 0.0, 0.0, 1.0034464485520185, 0.00921859...","[0.0, 0.0, 0.0, 0.19, 0.195, 0.19, 0.18, 0.18,...","[2.5518505225999373e-09, 2.5518505225999373e-0...","[0.0, 0.0, 0.0, 0.0300751879699248, 0.11027568..."
voice043,"[5.234080077272485e-36, 5.234080077272485e-36,...","[4.106449591680743e-35, 4.106449591680743e-35,...","[6.543559429669184e-35, 6.543559429669184e-35,...","[2.281356715613009e-34, 2.281356715613009e-34,...","[1.708916643375997e-36, 1.708916643375997e-36,...","[0.2, 0.2, 0.2, 0.0158092712796711, 0.00753253...","[2.6326916472224135e-36, 2.6326916472224135e-3...","[1.3823387765786216e-35, 1.3823387765786216e-3...","[3.565745472376315e-36, 3.565745472376315e-36,...","[9.461352370768469e-34, 9.461352370768469e-34,...",...,"[1.5423217051381124e-06, 1.5423217051381124e-0...","[1.219442233576635e-06, 1.219442233576635e-06,...","[1.1134746